# Training the mnist model

### Adapted from https://bensonruan.com/handwritten-digit-recognition-with-tensorflow-js/

In [1]:
# Import all packages needed
import tensorflow as tf
import tensorflowjs as tfjs
import keras as kr

Using TensorFlow backend.


In [2]:
# split the mnist data into train and test
(train_img,train_label),(test_img,test_label) = kr.datasets.mnist.load_data()

In [3]:
# reshape and scale the data
train_img = train_img.reshape([-1, 28, 28, 1])
test_img = test_img.reshape([-1, 28, 28, 1])
train_img = train_img/255.0
test_img = test_img/255.0


In [4]:
# convert class vectors to binary class matrices --> one-hot encoding
train_label = kr.utils.to_categorical(train_label)
test_label = kr.utils.to_categorical(test_label)

In [6]:
tf.compat.v1.disable_eager_execution()

In [7]:
model = kr.Sequential([
    kr.layers.Conv2D(32, (5, 5), padding="same", input_shape=[28, 28, 1]),
    kr.layers.MaxPool2D((2,2)),
    kr.layers.Conv2D(64, (5, 5), padding="same"),
    kr.layers.MaxPool2D((2,2)),
    kr.layers.Flatten(),
    kr.layers.Dense(1024, activation='relu'),
    kr.layers.Dropout(0.2),
    kr.layers.Dense(10, activation='softmax')
])




Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [13]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 28, 28, 32)        832       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 14, 14, 64)        51264     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 7, 7, 64)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 3136)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 1024)              3212288   
_________________________________________________________________
dropout_2 (Dropout)          (None, 1024)              0         
__________

In [9]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [10]:
model.fit(train_img,train_label, validation_data=(test_img,test_label), epochs=6)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 60000 samples, validate on 10000 samples
Epoch 1/6





60000/60000 [==============================] - 201s 3ms/step - loss: 0.1178 - acc: 0.9642 - val_loss: 0.0652 - val_acc: 0.9788
Epoch 2/6
60000/60000 [==============================] - 202s 3ms/step - loss: 0.0553 - acc: 0.9833 - val_loss: 0.0396 - val_acc: 0.9861
Epoch 3/6
60000/60000 [==============================] - 200s 3ms/step - loss: 0.0431 - acc: 0.9875 - val_loss: 0.0615 - val_acc: 0.9850
Epoch 4/6
60000/60000 [==============================] - 199s 3ms/step - loss: 0.0434 - acc: 0.9882 - val_loss: 0.0503 - val_acc: 0.9878
Epoch 5/6
60000/60000 [==============================] - 199s 3ms/step - loss: 0.0376 - acc: 0.9903 - val_loss: 0.0502 - val_acc: 0.9895
Epoch 6/6
60000/60000 [==============================] - 199s 3ms/step - loss: 0.0321 - acc: 0.9917 - val_loss: 0.0442 - val_acc: 0.9901


In [14]:
test_loss,test_acc = model.evaluate(test_img, test_label)

10000/10000 [==============================] - 3s 333us/step


In [15]:
model.save("model-9901.h5")

In [12]:
tfjs.converters.save_keras_model(model, 'static/models')